In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.brusly_pd import clean_cprr, clean_pprr
from lib.match import (
    ThresholdMatcher, NoopIndex, JaroWinklerSimilarity
)
from lib.explore import print_df
from match.brusly_pd import add_uid_to_complaint, add_supervisor_uid_to_complaint

In [2]:
cprr = clean_cprr()
cprr

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/brusly_pd.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  sup = df.supervisor.str.replace(
../clean/brusly_pd.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "supervisor_rank"] = sup.loc[:, 0].str.lower()\


,charges,action,last_name,first_name,supervisor_rank,supervisor_first_name,supervisor_last_name,receive_year,receive_month,receive_day,...,occur_month,occur_day,suspension_start_year,suspension_start_month,suspension_start_day,suspension_end_year,suspension_end_month,suspension_end_day,agency,data_production_year
0,failure to report in a timely manner,first offense counseling,portier,brett,assistant chief,tom,southon,2019,5,3,...,5,3,,,,,,,Brusly PD,2020
1,conduct unbecoming of an officer,suspended without pay,johnson,david,lieutenant,jason,fontenot,2020,5,5,...,4,29,2020,5,13,2020,5,29,Brusly PD,2020


In [3]:
post = pd.read_csv('../data/clean/pprr_post_2020_11_06.csv')
post = post[post.agency == 'brusly pd']

dfa = cprr[["first_name", "last_name"]]
dfb = post[["uid", "first_name", "last_name"]].set_index("uid", drop=True)

matcher = ThresholdMatcher(dfa, dfb, NoopIndex(), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity()
})
print_df(matcher.get_sample_pairs())

                                                                first_name last_name
score_range pair_idx sim_score row_key                                              
1.00-0.95   0        1.0       1                                     david   johnson
                               311a373584f02b60ff5d902d7dc319e5      david   johnson
            1        1.0       0                                     brett   portier
                               fbc660d7fcef80a66e904f71b51bbced      brett   portier
